<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#Load-packages" data-toc-modified-id="Load-packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load packages</a></span></li><li><span><a href="#Load-dataset:-Groundwater-chemistry-South-Australia" data-toc-modified-id="Load-dataset:-Groundwater-chemistry-South-Australia-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load dataset: Groundwater chemistry South Australia</a></span></li></ul></li><li><span><a href="#Clustering" data-toc-modified-id="Clustering-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Clustering</a></span><ul class="toc-item"><li><span><a href="#Preprocess-the-data" data-toc-modified-id="Preprocess-the-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Preprocess the data</a></span></li><li><span><a href="#Clustering-with-DBSCAN" data-toc-modified-id="Clustering-with-DBSCAN-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Clustering with DBSCAN</a></span></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Classification</a></span><ul class="toc-item"><li><span><a href="#Prepare-dataset" data-toc-modified-id="Prepare-dataset-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Prepare dataset</a></span></li><li><span><a href="#Classify-with-Random-Forests" data-toc-modified-id="Classify-with-Random-Forests-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Classify with Random Forests</a></span></li><li><span><a href="#Nitrate-class-prediction-based-on-TDS-and-major-ions" data-toc-modified-id="Nitrate-class-prediction-based-on-TDS-and-major-ions-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Nitrate class prediction based on TDS and major ions</a></span></li></ul></li><li><span><a href="#Regression" data-toc-modified-id="Regression-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Regression</a></span><ul class="toc-item"><li><span><a href="#Nitrate-regression" data-toc-modified-id="Nitrate-regression-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Nitrate regression</a></span></li></ul></li><li><span><a href="#Dimensionality-reduction" data-toc-modified-id="Dimensionality-reduction-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Dimensionality reduction</a></span></li></ul></div>

# Introduction
This notebook shows a couple of examples of machine learning approaches, applied to a hydrochemistry dataset. There are many Python packages for machine learning. Within this notebook, we can only scratch the surface of what is possible with machine learning. Rather than focusing on ML techniques, we focus on different tasks for which machine learning can help:

- clustering
- classification
- regression
- dimensionality reduction

This notebook uses the scikit-learn package. It is a user friendly package that provides a comprehensive, well-documented introduction to machine learning, with comparisons between techniques and plenty of example code.
## Load packages

In [ ]:
# preamble
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# allow interactive figures in notebook
%matplotlib notebook

## Load dataset: Groundwater chemistry South Australia
The dataset we are using is a publicly available dataset of groundwater chemistry of South Australia.

[Gray, David J. and Bardwell, Nicole (2016) Hydrogeochemistry of South Australia: Data Release: Accompanying Notes. CSIRO, Australia. EP156116 34p](https://data.csiro.au/collections/collection/CIcsiro:17862v1)


In [ ]:
fname = 'CSH_SA.xlsx'
sheet = 'Data'
chem = pd.read_excel(fname,sheet)
chem

# Clustering
Clustering is dividing the data samples into groups, based on their properties, without having predefined labels. Some approaches require to specify how many groups or clusters are in the data, most approaches do not require this input, but have a parameter that controls the number of groups.

In this example we will cluster the dataset based on the Total Dissolved Solids (TDS) and Alkalinity (HCO3). Clustering algorithms can use many more variables or dimensions. We limit ourselves to two to make it easier to visualise.

## Preprocess the data
Machine learning or data analysis techniques often work better if data have a similar scale and are (close to) normally distributed. There are many approaches to preprocess and normalise or standardise data. In this case, we just do a log10 transform.
Most algorithms can deal with missing values, but you need to check the documentation to learn how missing values are dealt with. If you have sufficient samples, it is better to drop any samples from the analysis that have missing values.

In [ ]:
# only select TDS & HCO3, drop any rows with missing values
TDSHCO3 = chem[['TDSc_mgL','HCO3_mgL']].dropna() 
# plot TDS v HCO3 on log log scale
plt.figure()
plt.loglog(TDSHCO3['TDSc_mgL'],TDSHCO3['HCO3_mgL'],'.k',alpha=0.2)
plt.xlabel('TDS (mg/L)')
plt.ylabel('HCO3 (mg/L)')
# log10 transform of dataframe
TDSHCO3_l = np.log10(TDSHCO3) #log10 transform

In [ ]:
TDSHCO3_l

## Clustering with DBSCAN
There are many clustering algorithms. DBSCAN is easy to use and generally results in robust clusters. The number of clusters is controlled by the eps parameter; the smaller the number, the more clusters it returns. To avoid many clusters with a single datapoint, you can specify the minimum number of samples per cluster.

In [ ]:
# import DBSCAN from sklearn
from sklearn.cluster import DBSCAN
# convert to numpy array
X = np.array(TDSHCO3_l)
# remove -inf values
X = X[~np.isinf(X).any(axis=1)]
# call the DBSCAN function
db = DBSCAN(eps=0.1, min_samples=10).fit(X)
# array with cluster labels, -1 is no cluster
labels = db.labels_
# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

# plot TDS v HCO3, points colored according to cluster
unique_labels = set(labels)
core_samples_mask = np.zeros_like(labels, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
plt.figure()
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = labels == k

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(
        xy[:, 0],
        xy[:, 1],
        "o",
        markerfacecolor=tuple(col),
        markeredgecolor="k",
        markersize=4,
    )

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(
        xy[:, 0],
        xy[:, 1],
        "o",
        markerfacecolor=tuple(col),
        markeredgecolor="k",
        markersize=1,
    )

plt.title(f"Estimated number of clusters: {n_clusters_}\nEstimated number of noise points: {n_noise_}")
plt.xlabel('TDS (mg/L)')
plt.ylabel('HCO3 (mg/L)')
plt.show()

# Classification
Clustering is similar to classification; the goal is to assign a sample to a group based on its properties. The main difference is that clustering is unsupervised (the algorithm seeks groups that are most similar within the group and most disimilar between groups), while classification is supervised. There is a training dataset with labelled samples. The algorithm tries to learn how to label the data and then predict what group new samples belong to.

In this example, we create 4 groups in the dataset, based on salinity:
- fresh (TDS < 1000 mg/L)
- slightly saline (1000 mg/L < TDS < 3000 mg/L)
- moderately saline (3000 mg/L < TDS < 10000 mg/L)
- highly saline (TDS > 10.000 mg/L)

We will train the algorithm to use the major ion concentrations (HCO3, Na, K, Mg, Ca, Cl, SO4, NO3) to predict the salinity class.

## Prepare dataset
The first task is to classify the samples. I've used a helper function for this. Plotting TDS v labels helps in verifying the classification is correct.

In [ ]:
# select variables
sal = chem[['TDSc_mgL','HCO3_mgL','Na_mgL','K_mgL','Mg_mgL','Ca_mgL','Cl_mgL','SO4_mgL','NO3N_mgL']].dropna()
# create labels based on TDS
def classifier(row):
    if row["TDSc_mgL"] <= 1000:
        return 1
    elif (row["TDSc_mgL"] > 1000) and (row["TDSc_mgL"] <= 3000):
        return 2
    elif (row["TDSc_mgL"] > 3000) and (row["TDSc_mgL"] <= 10000):
        return 3
    else:
        return 4
sal['class'] = sal.apply(classifier, axis=1)
# check labels
plt.figure()
plt.semilogx(sal['TDSc_mgL'],sal['class'],'.k')
plt.xlabel('TDS (mg/L)')
plt.ylabel('Label')
plt.yticks(np.arange(1,5),['Fresh','Slightly saline','Moderately saline', 'Highly saline'])
plt.tight_layout()

While salinity is a function of the total concentration of dissolved elements in the sample, it is not trivial to predict the salinity class based on major ion composition. If we plot the log concentrations of the ions against the labels, you'll see that there is no single element that can be used for the classification.

In [ ]:
plt.figure(figsize=(7,7))
for i,ion in enumerate(['HCO3_mgL','Na_mgL','K_mgL','Mg_mgL','Ca_mgL','Cl_mgL','SO4_mgL','NO3N_mgL']):
    plt.subplot(3,3,i+1)
    plt.semilogx(sal[ion],sal['class'],'.k')
    plt.title(ion)
plt.tight_layout()

## Classify with Random Forests
One of the first approaches for classification were decision trees. The algorithm tries to find ways to hierarchically split the datasets based on the properties. The approach works well, but is very sensitive to outliers. Ensemble methods have proven to be much more robust. A large number of decision trees are created by bootstrapping the data (resampling with replacement). The ensemble of decision trees is called a random forest.

To train a random forest classifier, we need to split the dataset into a training dataset and a validation dataset. Here I chose to use the first 4500 samples to train the classifier. The remainder of the dataset (~1000 samples) is used for validation.

In [ ]:
# import Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
# select the relevant variables from the dataframe and create numpy arrays
X = np.array(sal[['HCO3_mgL','Na_mgL','K_mgL','Mg_mgL','Ca_mgL','Cl_mgL','SO4_mgL','NO3N_mgL']])
Y = np.array(sal['class'])
# divide into training and validation
X_tr = X[0:4500,:]
Y_tr = Y[0:4500]
X_val = X[4500::,:]
Y_val = Y[4500::]
# create and train RandomForestClassifier (n_estimators is number of trees; more is better, but slower)
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_tr, Y_tr)
# test performance by predicting salinity class for validation samples
val = clf.predict(X_val)
# compute a summary table with the count of correctly and incorrectly classified samples
conf = np.zeros((4,4))
for i in np.arange(1,5):
    for j in np.arange(1,5):
        conf[i-1,j-1]=np.sum((Y_val==i)&(val==j))
conff = pd.DataFrame(conf,columns=['RF fresh','RF slightly saline','RF moderately saline', 'RF highly saline'],
                     index=['Fresh','Slightly saline','Moderately saline', 'Highly saline'])
conff

## Nitrate class prediction based on TDS and major ions
Let's try to do a similar exercise, but now label the dataset based on the nitrate concentration and try to predict the class based on TDS and major ions. The classes for nitrate are:
- No NO3 (NO3 < 1 mg/L)
- Low NO3 (1 mg/L < NO3 < 50 mg/L)
- High NO3 (NO3 > 50 mg/L)

In [ ]:
# create labels based on Nitrate
def classifier(row):
    if row["NO3N_mgL"] <= 1:
        return 1
    elif (row["NO3N_mgL"] > 1) and (row["NO3N_mgL"] <= 25):
        return 2
    else:
        return 3
sal['classNO3'] = sal.apply(classifier, axis=1)
# check labels
plt.figure()
plt.semilogx(sal['NO3N_mgL'],sal['classNO3'],'.k')
plt.xlabel('NO3 (mg/L)')
plt.ylabel('Label')
plt.yticks(np.arange(1,4),['No NO3','Low NO3','High NO3'])
plt.tight_layout()

In [ ]:
plt.figure(figsize=(7,7))
for i,ion in enumerate(['TDSc_mgL','HCO3_mgL','Na_mgL','K_mgL','Mg_mgL','Ca_mgL','Cl_mgL','SO4_mgL']):
    plt.subplot(3,3,i+1)
    plt.semilogx(sal[ion],sal['classNO3'],'.k')
    plt.title(ion)
plt.tight_layout()

In [ ]:
X = np.array(sal[['TDSc_mgL','HCO3_mgL','Na_mgL','K_mgL','Mg_mgL','Ca_mgL','Cl_mgL','SO4_mgL']])
Y = np.array(sal['classNO3'])
X_tr = X[0:4500,:]
Y_tr = Y[0:4500]
X_val = X[4500::,:]
Y_val = Y[4500::]
clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_tr, Y_tr)
val = clf.predict(X_val)
# compute a summary table with the count of correctly and incorrectly classified samples
conf = np.zeros((3,3))
for i in np.arange(1,4):
    for j in np.arange(1,4):
        conf[i-1,j-1]=np.sum((Y_val==i)&(val==j))
conff = pd.DataFrame(conf,columns=['RF no NO3','RF low NO3','RF high NO3'],
                     index=['no NO3','low NO3','high NO3'])
conff

From the table above it is clear that the classifier is not performing well. We can probably improve the classifier by using more samples, tweak the parameters or select a different algorithm. However, from a process understanding it is not surprising that predicting nitrate class from major ions does not work well - nitrate concentrations are controlled by landuse, vegetation and redox conditions. Major ions do not have information on those aspects.
# Regression
Instead of predicting classes of salinity, let's try to predict salinity values based on major ions. The random forest approach can also be used for regression.

In [ ]:
# import regressor
from sklearn.ensemble import RandomForestRegressor
# import function to calculate R2 score
from sklearn.metrics import r2_score

# prepare dataset again and divide in training and validation
X = np.array(sal[['HCO3_mgL','Na_mgL','K_mgL','Mg_mgL','Ca_mgL','Cl_mgL','SO4_mgL','NO3N_mgL']])
Y = np.log10(np.array(sal['TDSc_mgL']))
X_tr = X[0:4500,:]
Y_tr = Y[0:4500]
X_val = X[4500::,:]
Y_val = Y[4500::]

# create and train RF regressor
clf = RandomForestRegressor(n_estimators=50)
clf = clf.fit(X_tr, Y_tr)
# predict salinity for validation samples
val = clf.predict(X_val)
r2 = r2_score(Y_val,val)

# plot result
plt.figure()
plt.loglog(10**Y_val,10**val,'.k')
plt.title(f'Predict TDS from major ions \n R2 = {r2:4.3f}')
plt.xlabel('Measured TDS (mg/L)')
plt.ylabel('Modelled TDS (mg/L)')

## Nitrate regression
Let's try the same but predicting NO3 values from major ions

In [ ]:
from sklearn.ensemble import RandomForestRegressor

X = np.array(sal[['TDSc_mgL','HCO3_mgL','Na_mgL','K_mgL','Mg_mgL','Ca_mgL','Cl_mgL','SO4_mgL']])
Y = np.array(sal['NO3N_mgL'])
X_tr = X[0:4500,:]
Y_tr = Y[0:4500]
X_val = X[4500::,:]
Y_val = Y[4500::]

clf = RandomForestRegressor(n_estimators=50)
clf = clf.fit(X_tr, Y_tr)
val = clf.predict(X_val)
r2 = r2_score(Y_val,val)

plt.figure()
plt.plot(Y_val,val,'.k')
plt.title(f'Predict NO3 from major ions \n R2 = {r2:4.3f}')
plt.xlabel('Measured NO3 (mg/L)')
plt.ylabel('Modelled NO3 (mg/L)')

# Dimensionality reduction
Visualising high-dimensional data is very challenging. Dimensionality reduction can help by creating a projection of the data, where samples that are similar are plotted close to each other and samples that are very dissimilar are plotted far apart.
In this example, we'll create a projection of the major ion data, color each data point based on its position and visualise that on a map. I find this kind of visualisation useful when dealing with data that represent mixtures of end members, where it is not often possible to define clear clusters.

In [ ]:
cols = ['Lat','Long','TDSc_mgL', 'pH', 'HCO3_mgL', 'Na_mgL', 'K_mgL', 'Mg_mgL', 'Ca_mgL', 'Cl_mgL', 'SO4_mgL','NO3N_mgL']
dat = chem[cols].dropna() 

Rather than doing a log transform, I've preprocessed the data by taking the rank i.e. the position if you were to rank them from smallest to largest. This can be easily done wiith the `rankdata` function from [scipy.stats](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.rankdata.html).

In [ ]:
#import ranking function
from scipy.stats import rankdata
# calculate rank of data
dr = rankdata(np.array(dat[cols[2::]]),axis=0)

Here we'll look into dimensionality reduction and manifold learning. The goal is to find a representation of the data in 2D such that samples that are similar are plotted close to each other and samples that are very different are plotted far apart. The [manifold learning page](https://scikit-learn.org/stable/modules/manifold.html#manifold) gives an overview of methods you can use. The method we'll be using is [t-distributed Stochastic Neighbor Embedding](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE).

We need to import the function from scikit learn, specify the parameters and then train the algorithm with our dataset:

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, init='pca')
X = tsne.fit_transform(dr)

The result is a 2D numpy array with an x and y coordinate for each sample. We can visualise this with `plt.scatter` and color the plot with the rank of each of the variables:

In [ ]:
fig = plt.figure(figsize=(7,7))
for i,col in enumerate(cols[2::]):
    ax = plt.subplot(4,3,i+1, aspect=1)
    ax.scatter(X[:,0],X[:,1],0.5,dr[:,i],cmap='viridis')
    ax.set_title(col)
    ax.set_axis_off()
plt.tight_layout()

This is a spatial dataset, so we want to know show this information on a map. I've developed a 2D perceptually colormap that can be used to assign a unique color to each sample, based on the coordinates of the TSNE projection:

In [ ]:
def percuniform_rgb(x,y):
    '''
    Create RGB values for x,y positions from perceptually uniform colour scheme
    IN:
        x: [nx1] array of x values
        y: [nx1] array of y values
    OUT:
        rgb: [nx3] array of rgb values
    '''
    # rescale cartesian coordinates into range [-1,1]
    # normalise based on max(range(x),range(y))
    # multiply by 2 and subtract 1 to have data 
    # - centered on [0,0] 
    # - x and y each in range [-1,1]
    range_x = x.max()-x.min()
    range_y = y.max()-y.min()
    range_m = max(range_x,range_y)
    x_s = 2*((x-x.min())/range_m)-1
    y_s = 2*((y-y.min())/range_m)-1
    # load spline interpolant of colour scheme
    rgb_interp = np.load('BivariateColourScheme.npy', allow_pickle=True, encoding='latin1').item()
    # interpolate rgb values
    rgb = np.zeros((len(x),3))
    for i,col in enumerate(['R','G','B']):
        rgb[:,i] = np.clip(rgb_interp[col].ev(x_s,y_s),0,1)
    return(rgb)

In [ ]:
# assign color based on x,y coordinate in projected space
tsnergb = percuniform_rgb(X[:,0],X[:,1])
# create plot
fig,ax = plt.subplots()
ax.scatter(X[:,0],X[:,1],5,tsnergb)
ax.set_aspect('equal')
ax.set_title('Color based on position')
ax.set_axis_off()

We can now make a map of the samples, where each sample is colored based on its location in the TSNE plot

In [ ]:

fig,ax = plt.subplots()
# modify the default figure size
fig.set_size_inches(7,7)
ax.set_facecolor("whitesmoke")
ax.set_title('Color based on TSNE projection')
# zoom in to SA data
ax.set_xlim(chem['Long'].min(),chem['Long'].max())
ax.set_ylim(chem['Lat'].min(),chem['Lat'].max())
# use scatter to plot point with color based on recharge
s = ax.scatter(dat['Long'],dat['Lat'],15,tsnergb)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')